In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os



# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
my_string = 'ISIC_0000018_downsampled.jpg'
my_string[:12]
my_string[-4:]
new_name = my_string[:12] + my_string[-4:]

In [ ]:
meta_data = pd.read_csv('../input/isic-2019/ISIC_2019_Training_Metadata.csv')

In [ ]:
meta_data.columns

In [ ]:
meta_data.head()

In [ ]:
ground_truth_table = pd.read_csv('../input/isic-2019/ISIC_2019_Training_GroundTruth.csv')

In [ ]:
ground_truth_table.columns

In [ ]:
ground_truth_table.tail()

In [ ]:
ground_truth_table.count()

In [ ]:
ground_truth_table['image'] += '.jpg' 

In [ ]:
ground_truth_table['image']

In [ ]:
for root, dirs, filenames in os.walk('/kaggle/input/isic-2019/ISIC_2019_Training_Input/ISIC_2019_Training_Input/'):
    for filename in filenames:        
        if '_downsampled' in filename:
            index = ground_truth_table[ground_truth_table['image'] == filename[:12] + filename[-4:] ].index.tolist()
            ground_truth_table['image'][index] = filename
            
            #os.rename(os.path.join(root), os.path.join(root, filename[:12] + filename[-4:] + ".png"))
            

In [ ]:
index = ground_truth_table[ground_truth_table['image'] == 'ISIC_0073251.png' ].index.tolist()
ground_truth_table['image'][index] = 'ISIC_0073251_downsampled.png'

In [ ]:
import tensorflow as tf
from keras_preprocessing.image import ImageDataGenerator

In [ ]:
datagen=ImageDataGenerator(rescale=1./255.,validation_split=0.25)
train_generator  = datagen.flow_from_dataframe(
    dataframe = ground_truth_table,
    directory='/kaggle/input/isic-2019/ISIC_2019_Training_Input/ISIC_2019_Training_Input',
    x_col="image",
    y_col=['MEL', 'NV', 'BCC', 'AK', 'BKL', 'DF', 'VASC', 'SCC', 'UNK'],
    weight_col=None,
    target_size=(256, 256),
    color_mode="rgb",
    classes=None,
    class_mode="raw",
    batch_size=32,
    shuffle=False,
    seed=None,
    save_to_dir=None,
    save_prefix="",
    save_format="jpg",
    subset="training",
    interpolation="nearest",
    validate_filenames=False
)

valid_generator  = datagen.flow_from_dataframe(
    dataframe = ground_truth_table,
    directory='/kaggle/input/isic-2019/ISIC_2019_Training_Input/ISIC_2019_Training_Input',
    x_col="image",
    y_col=['MEL', 'NV', 'BCC', 'AK', 'BKL', 'DF', 'VASC', 'SCC', 'UNK'],
    weight_col=None,
    target_size=(256, 256),
    color_mode="rgb",
    classes=None,
    class_mode="raw",
    batch_size=32,
    shuffle=False,
    seed=None,
    save_to_dir=None,
    save_prefix="",
    save_format="jpg",
    subset="validation",
    interpolation="nearest",
    validate_filenames=False
)

In [ ]:
pwd

In [ ]:
os.listdir('/kaggle/input')

In [ ]:
from tensorflow.keras.layers import Conv2D, Activation, MaxPooling2D, Dropout,Flatten, Dense

model = tf.keras.Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=(256,256,3)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(9, activation='softmax'))


In [ ]:

model.compile(tf.keras.optimizers.RMSprop(lr=0.0001),
loss="categorical_crossentropy", metrics=["accuracy"])
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=10)

In [ ]:
k = os.listdir('/kaggle/input/isic-2019/ISIC_2019_Training_Input/ISIC_2019_Training_Input/')
#'ISIC_0057312.jpg'
#print(k)

In [ ]:
import matplotlib.pyplot as plt
hehe = plt.imread('/kaggle/input/isic-2019/ISIC_2019_Training_Input/ISIC_2019_Training_Input/ISIC_0057312.jpg')

In [ ]:
plt.imshow(hehe)